In [1]:
import matplotlib as mpl
import numpy as np

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten, Bidirectional, Merge, GRU
from keras.models import Sequential, load_model, save_model
from keras.optimizers import Adam

from sklearn import cluster
from sklearn import preprocessing as prep
from sklearn.model_selection import StratifiedKFold




Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [6]:
table = {'A':[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'C':[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'E':[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'D':[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'G':[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'F':[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'I':[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'H':[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         'K':[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
         'M':[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
         'L':[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
         'N':[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
         'Q':[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
         'P':[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
         'S':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
         'R':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
         'T':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
         'W':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
         'V':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
         'Y':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
         '_':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]}

result = []
resultLabel = []
#df = pd.read_csv("/Users/yimingzhao/Desktop/All_101.csv")
#df.columns = ['Protein accession','Position','Seq','Label']
data = open("/Users/yimingzhao/Desktop/All_101.csv").readlines()
print(data)
# seq_len = 2
# #data = "GGGCGGGGUAGCUUUGUAAAUGUUUUUCUGAUUAACAAUGUCUUCUCUUUUUUAGCUAUUACAGGGAGGAGUGUUGAGACCAGAUGUCAUCUACUGUCUCUUGGGUCAGCAGCACGCAUGACAGGACCAAGGAAUGGCAGCAACACGCAGAAUCUUAGCUAGCGCUCUCCAGCAGUCUUCUCUAUUAGGGAAUGUCUAAUUGGCAUGAGAUGUUCAAAUCAGACCUGGGAUUGGACAACUGUAAAGUAUGACUGGAUCGUCAGGGCGUCGCUUGGACUCUGUAAUCUAAUGUUUAGGGCAUAUUGAAGUUGAGGUGCUGCCUUCCAGAACUUAAACAUGUAGCUCACUUUCCCCCAUCUUUUUGACAAAUGACCAGUAGUUAAUUUCUAAGGUUUUUCUUAUCAAGAGAAGAAUACUUAAAAACUCUUCCUUGUUUGCAAAAGAAACUUCUGAGGACUGAGUCUUAACUAUGCACACAAUGUCACCAGACAUCUUGAAACU ENSMUST00000000001 37026 1"
# for i in range(len(data)):
#     cut = (len(data[i].split()[0]) - 1 - seq_len) // 2
#     print(cut)
#     trainData = data[i].split()[3][cut:-cut]
#     print(trainData)
#     trainLabel = int(data[i].split()[-1])
#     print(trainLabel)
#     trainDataArray = []
#     for j in range(len(trainData)):
#         trainDataArray.append(table[j])
# #         if trainData[j] == 'A':
# #             trainDataArray.append([1, 0, 0, 0])
# #         elif trainData[j] == 'C':
# #             trainDataArray.append([0, 1, 0, 0])
# #         elif trainData[j] == 'G':
# #             trainDataArray.append([0, 0, 1, 0])
# #         elif trainData[j] == 'U':
# #             trainDataArray.append([0, 0, 0, 1])
# #         else:
# #             trainDataArray.append([0, 0, 0, 0])
#     result.append(trainDataArray)
#     resultLabel.append(trainLabel)
# result = np.array(result)
# resultLabel = np.array(resultLabel)
# result = result.reshape((result.shape[0], -1, 1))
# print(result[0:3], resultLabel[0:3])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

